In [1]:
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import lightgbm as lgb
import gc
from joblib import Parallel, delayed
from util import *
import warnings
warnings.filterwarnings('ignore')

In [2]:
def label_encode(df, cols, verbose=True):
    """
    label encode
    @param df:
    @param cols:
    @param verbose:
    @return:
    """
    for col in cols:
        df[col], _ = df[col].astype(str).factorize(sort=True)
        if df[col].max() > 32000:
            df[col] = df[col].astype('int32')
        else:
            df[col] = df[col].astype('int16')
        if verbose:
            print(col)
    return df

In [3]:
# 数据集7：entprise_info.csv
# 带标注的企业数据。每一行代表一个企业，每一行2列，其中id列为企业唯一标识，label列为标注（1：有非法集资风险，0：无非法集资风险），列之间采用“,”分隔符分割。
# 训练集 id 及标签
entprise_info = pd.read_csv('../input/train/entprise_info.csv')
print(entprise_info.shape)
entprise_info.head()

(14865, 2)


,id,label
0,59b38c56de3836831ff90a77d892a13523b7494f6ed09ff7,1
1,da8691b210adb3f6be8064e006f220070565db287275ad38,0
2,82750f1b9d122350918121f97c99bf96e11aa24ee91504a9,0
3,f000950527a6feb6b2c6de6f85c1e7438ba5590be931e2ec,0
4,f1c1045b13d1832927e3743e49d2917f2d98424f0849a373,0


In [4]:
# 数据集8（验证集）：entprise_evaluate.csv
# 未标注企业数据。参赛队伍需提交的最终结果数据集，每一行代表一个企业，每一行有 2 列, 其中id列为企业唯一标识，score列为空，列之间采用“,”分隔符分割。
# 测试集 id score
entprise_evaluate = pd.read_csv('../input/entprise_evaluate.csv')

entprise_evaluate.rename(columns={'score': 'label'}, inplace=True)

print(entprise_evaluate.shape)
entprise_evaluate.head()

(10000, 2)


,id,label
0,82750f1b9d1223508ee329d47e27d35176c93eb9f35e9c1a,NaN
1,f000950527a6feb670cc1c87c2025f3922aaa4a0206a0a33,NaN
2,e9f7b28ec10e04700ef4db75a494f9a1e8e8b09555e6afa1,NaN
3,beb4aaaa89e0a0ae9d77bd5d7665be6342f552f51840cf19,NaN
4,e9f7b28ec10e0470ee4172cec0133b6826c34f27d3dff204,NaN


In [5]:
data = pd.concat([entprise_info, entprise_evaluate], axis=0, ignore_index=True)

In [6]:
data.shape

(24865, 2)

In [7]:
# 数据集1：base_info.csv
# 包含数据集7和8中涉及到的所有企业的基本信息，每一行代表一个企业的基本数据，每一行有33列，其中id列为企业唯一标识，列之间采用“,”分隔符分割。
# 数据格式如下：
# [id:企业唯一标识, oplocdistrict:行政区划代码, industryphy:行业类别代码, industryco:行业细类代码, dom:经营地址, opscope:经营范围, enttype:企业类型, enttypeitem:企业类型小类, opfrom:经营期限起, opto:经营期限止, state:状态, orgid:机构标识, jobid:职位标识, adbusign:是否广告经营, townsign:是否城镇, regtype:主题登记类型, empnum:从业人数, compform:组织形式, parnum:合伙人数, exenum:执行人数, opform:经营方式, ptbusscope:兼营范围, venind:风险行业, enttypeminu:企业类型细类, midpreindcode:中西部优势产业代码, protype:项目类型, oploc:经营场所, regcap:注册资本（金）, reccap:实缴资本, forreccap:实缴资本（外方）, forregcap:注册资本（外方）, congro:投资总额, enttypegb:企业（机构）类型]
# 基础信息表
base_info = pd.read_csv('../input/train/base_info.csv')
print(base_info.shape)
base_info.info()

(24865, 33)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24865 entries, 0 to 24864
Data columns (total 33 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             24865 non-null  object 
 1   oplocdistrict  24865 non-null  int64  
 2   industryphy    24865 non-null  object 
 3   industryco     24864 non-null  float64
 4   dom            24865 non-null  object 
 5   opscope        24865 non-null  object 
 6   enttype        24865 non-null  int64  
 7   enttypeitem    16651 non-null  float64
 8   opfrom         24865 non-null  object 
 9   opto           8825 non-null   object 
 10  state          24865 non-null  int64  
 11  orgid          24865 non-null  int64  
 12  jobid          24865 non-null  int64  
 13  adbusign       24865 non-null  int64  
 14  townsign       24865 non-null  int64  
 15  regtype        24865 non-null  int64  
 16  empnum         19615 non-null  float64
 17  compform       10631 non-null  float64

In [8]:
len(set(data['id']) & set(base_info['id']))

24865

In [9]:
single_cols = ['ptbusscope', 'midpreindcode']
many_cols = ['dom', 'opscope']
to_drop = single_cols + many_cols
base_info.drop(to_drop, axis=1, inplace=True)
gc.collect()

40

In [10]:
base_info['opform'] = base_info['opform'].replace('01', '01-以个人财产出资').replace('02', '02-以家庭共有财产作为个人出资').replace('   ', np.nan)

In [11]:
def identify_missing(df, missing_threshold):
    """
    缺失率
    @param df:
    @param missing_threshold:
    @return:
    """
    missing_rate = df.isnull().sum() / len(df)
    missing_rate = missing_rate.sort_values(ascending=False)
    print(missing_rate)
    to_drop = missing_rate[missing_rate > missing_threshold].index.to_list()
    print('{} features with greater than {} missing values.\n'.format(len(to_drop), missing_threshold))
    return to_drop

In [12]:
to_drop = identify_missing(base_info, missing_threshold=0.5)
base_info.drop(to_drop, axis=1, inplace=True)
to_drop

protype          0.998633
forreccap        0.990871
congro           0.989986
forregcap        0.989946
exenum           0.944581
parnum           0.905932
reccap           0.715102
enttypeminu      0.707621
venind           0.660688
opto             0.645083
opform           0.638086
compform         0.572451
enttypeitem      0.330344
empnum           0.211140
regcap           0.007681
industryco       0.000040
oplocdistrict    0.000000
industryphy      0.000000
enttype          0.000000
opfrom           0.000000
enttypegb        0.000000
state            0.000000
orgid            0.000000
jobid            0.000000
adbusign         0.000000
townsign         0.000000
regtype          0.000000
oploc            0.000000
id               0.000000
dtype: float64
12 features with greater than 0.5 missing values.



['protype',
 'forreccap',
 'congro',
 'forregcap',
 'exenum',
 'parnum',
 'reccap',
 'enttypeminu',
 'venind',
 'opto',
 'opform',
 'compform']

In [13]:
num_cols = [i for i in ['empnum', 'parnum', 'exenum', 'regcap', 'reccap', 'forreccap', 'forregcap', 'congro'] if i not in to_drop]
le_cols = [i for i in ['oplocdistrict', 'industryphy', 'industryco', 'enttype', 'enttypeitem', 'state',
           'orgid', 'jobid', 'regtype', 'compform', 'opform', 'venind', 'enttypeminu',
           'protype', 'oploc', 'enttypegb'] if i not in to_drop]
one_zero_cols = ['adbusign', 'townsign']
cat_cols = le_cols + one_zero_cols
# dt_cols = ['opfrom', 'opto']

In [14]:
# 时间转换, 暂时先抽取年份特征
base_info['opfrom'] = pd.to_datetime(base_info.opfrom)
base_info['opfrom_year'] = base_info['opfrom'].dt.year.astype('int')

# base_info['opto'] = pd.to_datetime(base_info.opto)
# base_info['opto_year'] = base_info['opto'].dt.year.fillna(-1).astype('int')

del base_info['opfrom']
# del base_info['opto']
gc.collect()

12

In [15]:
le_cols += ['opfrom_year']

In [16]:
base_info = label_encode(base_info, le_cols, verbose=True)

oplocdistrict
industryphy
industryco
enttype
enttypeitem
state
orgid
jobid
regtype
oploc
enttypegb
opfrom_year


In [17]:
# 划分训练集和测试集
entprise_evaluate.columns = ['id', 'label']
data = pd.concat([entprise_info, entprise_evaluate])
df = pd.merge(data, base_info, on='id', how='left')

print(df.shape)
df.head()

(24865, 18)


,id,label,oplocdistrict,industryphy,industryco,enttype,enttypeitem,state,orgid,jobid,adbusign,townsign,regtype,empnum,oploc,regcap,enttypegb,opfrom_year
0,59b38c56de3836831ff90a77d892a13523b7494f6ed09ff7,1.0,3,9,122,9,17,4,32,150,0,1,0,2.0,108,42205.0,38,30
1,da8691b210adb3f6be8064e006f220070565db287275ad38,0.0,6,14,240,16,31,4,46,175,0,1,0,2.0,2547,10.0,52,33
2,82750f1b9d122350918121f97c99bf96e11aa24ee91504a9,0.0,7,12,206,0,2,4,40,424,0,0,0,3.0,108,100.0,4,34
3,f000950527a6feb6b2c6de6f85c1e7438ba5590be931e2ec,0.0,3,12,198,0,4,4,1,367,0,0,0,3.0,108,1285.0,6,33
4,f1c1045b13d1832927e3743e49d2917f2d98424f0849a373,0.0,12,14,245,16,31,4,70,404,0,1,0,2.0,2291,100.0,52,34


In [18]:
train = df[df.label.notna()]
test = df[df.label.isna()]

print(train.shape, test.shape)

(14865, 18) (10000, 18)


In [19]:
y = train['label'].astype(int)
sub = test[['id']]
used_cols = [i for i in train.columns if i not in ['id', 'label']]
train = train[used_cols]
test = test[used_cols]

In [20]:
X_train, X_valid, y_train, y_valid = train_test_split(train, y, test_size=0.25, random_state=2020)

cols = X_train.columns
useful_dict, useless_dict, useful_cols, useless_cols = auc_select(X_train, y_train, X_valid, y_valid, cols, threshold=0.52)
print('AUC useless_cols: \n', useless_cols)

oplocdistrict
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[8]	training's auc: 0.800172	valid_0's auc: 0.784394
Evaluated only: auc
**********
0.784394131780055
industryphy
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[33]	training's auc: 0.977366	valid_0's auc: 0.967944
Evaluated only: auc
**********
0.9679441641137959
industryco
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[104]	training's auc: 0.986598	valid_0's auc: 0.975303
Evaluated only: auc
**********
0.9753030403156906
enttype
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[52]	training's auc: 0.866492	valid_0's auc: 0.846703
Evaluated only: auc
**********
0.8467026664715628
enttypeitem
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[14]	training's auc: 0.910801	valid_0's auc: 0.8

In [21]:
X_train = X_train[useful_cols]
X_valid = X_valid[useful_cols]
test = test[useful_cols]

In [22]:
cols = X_train.columns.to_list()
print('Final cols: \n', len(cols))
train_dataset = lgb.Dataset(X_train, y_train)
valid_dataset = lgb.Dataset(X_valid, y_valid, reference=train_dataset)
all_dataset = lgb.Dataset(train[cols], y, reference=train_dataset)


def lgb_f1_score(y_hat, data):
    y_true = data.get_label()
    y_hat = np.where(y_hat > 0.5, 1, 0)  
    return 'f1', f1_score(y_true, y_hat), True


def f1_loss(y, pred):
    beta = 2
    p = 1. / (1 + np.exp(-pred))
    grad = p * ((beta - 1) * y + 1) - beta * y
    hess = ((beta - 1) * y + 1) * p * (1.0 - p)
    return grad, hess


params = {
    'objective': 'binary',
    'boosting': 'gbdt',
    'metric': 'auc',
#     'metric': 'None',  # 用自定义评估函数是将metric设置为'None'
    'learning_rate': 0.1,
    'num_leaves': 31,
    'lambda_l1': 0,
    'lambda_l2': 1,
    'num_threads': 23,
    'min_data_in_leaf': 20,
    'first_metric_only': True,
    'is_unbalance': True,
    'max_depth': -1,
    'seed': 2020
}

valid_model = lgb.train(
    params,
    train_dataset,
    valid_sets=[train_dataset, valid_dataset],
    early_stopping_rounds=200,
    verbose_eval=300,
#     feval=lgb_f1_score,
#     fobj=f1_loss
    
    
)
pred = valid_model.predict(X_valid)
y_valid_pred = np.where(pred > 0.5, 1, 0)
F1 = np.round(f1_score(y_valid, y_valid_pred), 5)
print('Valid F1: ', F1)
print('Valid mean label: ', np.mean(y_valid_pred))

train_model = lgb.train(
    params,
    all_dataset,
    num_boost_round=valid_model.best_iteration+20,
#     feval=lgb_f1_score,
#     fobj=f1_loss
)
y_test_pred = np.where(train_model.predict(test) > 0.5, 1, 0)

print('Test mean label: ', np.mean(y_test_pred))
sub['score'] = y_test_pred
sub.to_csv('../sub/baseline_{}_{}.csv'.format(time.strftime('%Y%m%d'), str(F1)), index=False)

Final cols: 
 14
Training until validation scores don't improve for 200 rounds
Did not meet early stopping. Best iteration is:
[100]	training's auc: 0.998773	valid_1's auc: 0.98725
Evaluated only: auc
Valid F1:  0.78417
Valid mean label:  0.08393866020984665
Test mean label:  0.1338
